**Want to add your first tool?** LLMs can't natively do arithmetic (beyond simple arithmetic with small numbers), so let's give our model a tool for doing arithmetic! 😎

Full example: [quickstart_with_math_tool.py](https://github.com/Rhobota/lasagna-ai/blob/main/examples/quickstart_with_math_tool.py)

In [1]:
# 🚧 Under Construction 🏗️

### Demo: Parallel Tool-calling

Because Lasagna AI is so deeply built on `asyncio`, if the Model asks for more than one tool call, then Lasagna will call those tools _in parallel_! This provides a speed boost, but keep this in mind so that you manage state correctly (i.e. no race conditions).